In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d shayanfazeli/

Dataset URL: https://www.kaggle.com/datasets/shayanfazeli/heartbeat
License(s): unknown
 95% 94.0M/98.8M [00:01<00:00, 110MB/s]
100% 98.8M/98.8M [00:01<00:00, 94.3MB/s]


In [ ]:
!unzip /content/heartbeat.zip

Archive:  /content/heartbeat.zip
  inflating: mitbih_test.csv         
  inflating: mitbih_train.csv        
  inflating: ptbdb_abnormal.csv      
  inflating: ptbdb_normal.csv        


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

In [ ]:
#df = pd.read_csv("/content/mitbih_train.csv")
df = pd.read_csv("/content/mitbih_test.csv")

In [ ]:
df

,1.000000000000000000e+00,7.582644820213317871e-01,1.115702465176582336e-01,0.000000000000000000e+00,8.057851344347000122e-02,7.851240038871765137e-02,6.611569970846176147e-02,4.958677664399147034e-02,4.752065986394882202e-02,3.512396663427352905e-02,...,0.000000000000000000e+00.56,0.000000000000000000e+00.57,0.000000000000000000e+00.58,0.000000000000000000e+00.59,0.000000000000000000e+00.60,0.000000000000000000e+00.61,0.000000000000000000e+00.62,0.000000000000000000e+00.63,0.000000000000000000e+00.64,0.000000000000000000e+00.65
0,0.908425,0.783883,0.531136,0.362637,0.366300,0.344322,0.333333,0.307692,0.296703,0.300366,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.730088,0.212389,0.000000,0.119469,0.101770,0.101770,0.110619,0.123894,0.115044,0.132743,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.910417,0.681250,0.472917,0.229167,0.068750,0.000000,0.004167,0.014583,0.054167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.570470,0.399329,0.238255,0.147651,0.000000,0.003356,0.040268,0.080537,0.070470,0.090604,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.000000,0.923664,0.656489,0.195929,0.111959,0.175573,0.122137,0.050891,0.035623,0.055980,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21886,0.928736,0.871264,0.804598,0.742529,0.650575,0.535632,0.394253,0.250575,0.140230,0.102299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
21887,0.802691,0.692078,0.587444,0.446936,0.318386,0.189836,0.118087,0.077728,0.112108,0.152466,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
21888,1.000000,0.967359,0.620178,0.347181,0.139466,0.089021,0.103858,0.100890,0.106825,0.100890,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
21889,0.984127,0.567460,0.607143,0.583333,0.607143,0.575397,0.575397,0.488095,0.392857,0.238095,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [ ]:
df.head()

,1.000000000000000000e+00,7.582644820213317871e-01,1.115702465176582336e-01,0.000000000000000000e+00,8.057851344347000122e-02,7.851240038871765137e-02,6.611569970846176147e-02,4.958677664399147034e-02,4.752065986394882202e-02,3.512396663427352905e-02,...,0.000000000000000000e+00.56,0.000000000000000000e+00.57,0.000000000000000000e+00.58,0.000000000000000000e+00.59,0.000000000000000000e+00.60,0.000000000000000000e+00.61,0.000000000000000000e+00.62,0.000000000000000000e+00.63,0.000000000000000000e+00.64,0.000000000000000000e+00.65
0,0.908425,0.783883,0.531136,0.362637,0.366300,0.344322,0.333333,0.307692,0.296703,0.300366,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.730088,0.212389,0.000000,0.119469,0.101770,0.101770,0.110619,0.123894,0.115044,0.132743,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.910417,0.681250,0.472917,0.229167,0.068750,0.000000,0.004167,0.014583,0.054167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.570470,0.399329,0.238255,0.147651,0.000000,0.003356,0.040268,0.080537,0.070470,0.090604,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.000000,0.923664,0.656489,0.195929,0.111959,0.175573,0.122137,0.050891,0.035623,0.055980,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(f'rows : {(df.shape[0])}')
print(f'columns : {(df.shape[1])}')

rows : 21891
columns : 188


In [ ]:
df.isnull().sum().any()

False

In [ ]:
df.iloc[:,187].value_counts()

,count
0.000000000000000000e+00.65,
0.0,18117
4.0,1608
2.0,1448
1.0,556
3.0,162


In [ ]:
# Separate features (X) and target (y)
X = df.iloc[:, :-1]  # All columns except the last one
y = df.iloc[:, -1]   # Last column as the target

# Step 3: Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 4: Scale the data (Normalization)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 5: Balance the data using SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)

# Step 6: Apply PCA to reduce dimensionality
pca = PCA(n_components=0.95)  # Retain 95% of variance
X_train_pca = pca.fit_transform(X_train_balanced)
X_test_pca = pca.transform(X_test_scaled)

# Step 7: Train the classifier (Random Forest in this case)
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train_pca, y_train_balanced)

# Step 8: Make predictions
y_pred = classifier.predict(X_test_pca)

# Step 9: Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98      3624
         1.0       0.78      0.68      0.72       111
         2.0       0.92      0.81      0.86       290
         3.0       0.51      0.66      0.58        32
         4.0       0.99      0.94      0.97       322

    accuracy                           0.96      4379
   macro avg       0.84      0.81      0.82      4379
weighted avg       0.96      0.96      0.96      4379



In [ ]:
#import tensorflow as tf
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense

# Define the model
#model = Sequential([
 #   Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
  #  Dense(32, activation='relu'),
   # Dense(len(y.unique()), activation='softmax')  # Use 'softmax' for multi-class classification
#])

# Compile the model
#model.compile(optimizer='adam',
 #             loss='sparse_categorical_crossentropy',  # Use 'sparse_categorical_crossentropy' if y is integer encoded
  #            metrics=['accuracy'])

# Train the model
#history = model.fit(X_train, y_train, epochs=30, validation_split=0.1, batch_size=32)


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98      3624
         1.0       0.78      0.68      0.72       111
         2.0       0.92      0.81      0.86       290
         3.0       0.51      0.66      0.58        32
         4.0       0.99      0.94      0.97       322

    accuracy                           0.96      4379
   macro avg       0.84      0.81      0.82      4379
weighted avg       0.96      0.96      0.96      4379



In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.utils import to_categorical
from sklearn.metrics import classification_report

# Reshape data for 1D CNN
X_train_pca = X_train_pca[..., np.newaxis]
X_test_pca = X_test_pca[..., np.newaxis]

# Ensure y_train_balanced and y_test are one-hot encoded
y_train_balanced = to_categorical(y_train_balanced)
y_test = to_categorical(y_test)

# Build the CNN model
model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_pca.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(y_train_balanced.shape[1], activation='softmax')  # Number of classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_pca, y_train_balanced, epochs=10, batch_size=32, validation_split=0.1)

# Make predictions
y_pred = model.predict(X_test_pca)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Print classification report
print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes))


Epoch 1/10
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - accuracy: 0.8247 - loss: 0.4831 - val_accuracy: 0.9383 - val_loss: 0.1946
Epoch 2/10
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.9540 - loss: 0.1389 - val_accuracy: 0.9830 - val_loss: 0.0571
Epoch 3/10
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9728 - loss: 0.0848 - val_accuracy: 0.9981 - val_loss: 0.0105
Epoch 4/10
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9815 - loss: 0.0594 - val_accuracy: 0.9938 - val_loss: 0.0260
Epoch 5/10
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9856 - loss: 0.0475 - val_accuracy: 0.9934 - val_loss: 0.0240
Epoch 6/10
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9884 - loss: 0.0371 - val_accuracy: 0.9977 - val_loss: 0.0070
Epoch 7/10
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step - accuracy: 0.9894 - loss: 0.0335 - val_accuracy: 0.9957 - val_loss: 0.0110
Epoch 8/10
2039/2039 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9911 - loss: 0.0304 